In [23]:
from docx import Document
import pandas as pd

def merge_all_tables_to_dataframe(file_path):
    # Load the Word document
    doc = Document(file_path)
    
    # Initialize an empty list to store DataFrames
    all_tables = []
    
    # Loop through all tables in the document
    for table in doc.tables:
        # Extract data from the table
        data = []
        for row in table.rows:
            # Remove \n and strip each cell's text
            data.append([cell.text.replace('\n', ' ').strip() for cell in row.cells])
        
        # Convert the table data to a DataFrame
        df = pd.DataFrame(data)
        
        # Use the first row as the header if applicable
        df.columns = df.iloc[0]  # Set the first row as the header
        df = df[1:]  # Drop the header row from data
        
        # Append the DataFrame to the list
        all_tables.append(df)
    
    # Concatenate all DataFrames into a single DataFrame
    merged_df = pd.concat(all_tables, ignore_index=True)
    
    return merged_df

# Example Usage
file_path = r"D:\ProvenTech\Document_2_Test.docx"  # Replace with the path to your Word document
dtf = merge_all_tables_to_dataframe(file_path)


In [24]:
dtf.columns=["Sr.","Test","Specification","Reference"]
dtf

,Sr.,Test,Specification,Reference
0,1.0,Description,White to Off white powder.,Current USP
1,2.0,Solubility,"Very soluble in water and in methanol, very s...",Current USP
2,3.0,A. Identification by IR B. Identification...,A. The infrared absorption spectrum of the sa...,Current USP
3,4.0,Water Content,Not more than 0.5% w/w,Current USP
4,5.0,Residue on ignition,Not more than 0.10% w/w,Current USP
5,6.0,Heavy metals,Not more than 0.001%,In-House
6,7.0,Tartaric acid content by Titrimetry (on anhyd...,Between 36.4% to 38.3% w/w,In-Hosue
7,8.0,Organic Impurities Procedure-1 A. Phenol Impu...,A. Not more than 0.10% B. Not more than 0.15% ...,Current USP
8,,Procedure-2 (Enantiomeric Purity) R-Enantiomer,Not more than 0.15%,Current USP
9,9.0,Assay by HPLC (on anhydrous basis),Between 98.0% to 102.0%,Current USP


In [25]:
dtf["Sr."]=range(1, len(dtf) + 1)
dtf

,Sr.,Test,Specification,Reference
0,1,Description,White to Off white powder.,Current USP
1,2,Solubility,"Very soluble in water and in methanol, very s...",Current USP
2,3,A. Identification by IR B. Identification...,A. The infrared absorption spectrum of the sa...,Current USP
3,4,Water Content,Not more than 0.5% w/w,Current USP
4,5,Residue on ignition,Not more than 0.10% w/w,Current USP
5,6,Heavy metals,Not more than 0.001%,In-House
6,7,Tartaric acid content by Titrimetry (on anhyd...,Between 36.4% to 38.3% w/w,In-Hosue
7,8,Organic Impurities Procedure-1 A. Phenol Impu...,A. Not more than 0.10% B. Not more than 0.15% ...,Current USP
8,9,Procedure-2 (Enantiomeric Purity) R-Enantiomer,Not more than 0.15%,Current USP
9,10,Assay by HPLC (on anhydrous basis),Between 98.0% to 102.0%,Current USP


In [26]:
SPEC_TABLE=dtf
SPEC_TABLE.to_csv("D:\ProvenTech\SPEC_TABLE_2.csv", index=False)

In [31]:
lst=df.iloc[2].tolist()
lst

['3.0',
 'A. Identification by IR      B. Identification by  HPLC',
 'A. The infrared absorption spectrum of  the sample in a  potassium bromide  dispersion exhibits maxima only at the  same wavelengths  as that of a similar  preparation of  Rivastigmine tartrate  Reference  Standard or Working  Standard.  B. The retention time of the major peak  of the sample solution corresponds to that  of system suitability solution, as obtained  in the test for organic impurities,  Procedure-2.',
 'Current USP']

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_groq import ChatGroq
llm = ChatGroq(
        model_name="llama-3.1-70b-versatile",
        temperature=0,
        groq_api_key="Your Groq API key"
    )
prompt_template = PromptTemplate.from_template(
    """You are an expert in deriving a list from another list based on given conditions.
    you are given a row of a dataframe in the form of list as input. the columns of this dataframe is [Sr., Test,Specification,Reference].
    you have to return a list so that I can append it to a new dataframe. the new dataframe columns are [TEST_NAME,SUB_TEST,TEXT_LIMIT,NLT(Not Less Than),NMT(Not More Than)]
    so keep in mind to return list of values in such a way that I can append to this dataframe.
    here are the conditions of convertion:
    TEST_NAME: This will be directly taken from the Test column in SPEC_TABLE.
    SUB_TEST: If a sub-test is present, populate this column with its name; otherwise, use the value from the Test column.
    TEXT_LIMIT: Extract this directly from the Specification column of SPEC_TABLE.
    NLT (Not Less Than): Extract this value from the Specification text where it states "not less than" and store it as a string.
    NMT (Not More Than): Similarly, extract this value where it states "not more than" and store it as a string.
    Handling "Between" Specifications:
    If the Specification mentions "between," you will encounter two values. Populate these values accordingly:
    The first value should be assigned to NLT,
    The second value should be assigned to NMT.
    If Specification value has more than one value then return those number of lists with seperated sub topics and mention those names in subtests.
    example for splitting: input:['8.0',
 'Organic Impurities Procedure-1  A. Phenol Impurity B. DPTTA C. Nor impurity D. Any other Impurity E. Total impurities',
 'A. Not more than 0.10% B. Not more than 0.15% C. Not more than 0.15% D. Not more than 0.10% E. Not more than 0.50%',
 'Current USP']
    output:[["Organic Impurities Procedure-1","Phenol Impurity","Not more than 0.10%","NLT extracted from specification","NMT extracted from specification"],
    ["Organic Impurities Procedure-1","DPTTA","Not more than 0.15%","NLT extracted from specification","NMT extracted from specification"],
    [same for c],[for D],[for E]]
    if NLT and NMT are not present just put None.
    input :{lst}
    output: only return list and nothing else.[] or [[],[],....]
    NO PREAMBLE
    return only list not code or any explanation
    """
)
chain = LLMChain(llm=llm, prompt=prompt_template)

In [37]:
columns = ["TEST_NAME", "SUB_TEST", "TEXT_LIMIT", "NLT (Not Less Than)", "NMT (Not More Than)"]
LIMITS = pd.DataFrame(columns=columns)

def append_to_dataframe(LIMITS, data):
    # Ensure data is a list of lists
    if isinstance(data[0], list):
        new_data = data  # Data is already a list of lists
    else:
        new_data = [data]  # Convert single row to a list of lists

    # Convert new data to a DataFrame and append
    new_df = pd.DataFrame(new_data, columns=LIMITS.columns)
    return pd.concat([LIMITS, new_df], ignore_index=True)



In [38]:
import ast
for index in range(len(SPEC_TABLE)):
    row = SPEC_TABLE.iloc[index].tolist()
    response = chain.run({"lst": row})
    actual_list = ast.literal_eval(response)
    LIMITS = append_to_dataframe(LIMITS, actual_list)


In [39]:
LIMITS

,TEST_NAME,SUB_TEST,TEXT_LIMIT,NLT (Not Less Than),NMT (Not More Than)
0,Description,Description,White to Off white powder.,None,None
1,Solubility,Solubility,"Very soluble in water and in methanol, very s...",None,None
2,Identification by IR,Identification by IR,None,None,None
3,Identification by HPLC,Identification by HPLC,None,None,None
4,Water Content,Water Content,Not more than 0.5% w/w,None,0.5% w/w
5,Residue on ignition,Residue on ignition,Not more than 0.10% w/w,None,0.10%
6,Heavy metals,Heavy metals,Not more than 0.001%,None,0.001%
7,Tartaric acid content by Titrimetry (on anhyd...,Tartaric acid content by Titrimetry (on anhyd...,Between 36.4% to 38.3% w/w,36.4% w/w,38.3% w/w
8,Organic Impurities Procedure-1,Phenol Impurity,Not more than 0.10%,None,0.10%
9,Organic Impurities Procedure-1,DPTTA,Not more than 0.15%,None,0.15%


In [40]:
LIMITS.to_csv("D:\ProvenTech\LIMITS_2.csv", index=False)

In [41]:
SPEC_TABLE

,Sr.,Test,Specification,Reference
0,1,Description,White to Off white powder.,Current USP
1,2,Solubility,"Very soluble in water and in methanol, very s...",Current USP
2,3,A. Identification by IR B. Identification...,A. The infrared absorption spectrum of the sa...,Current USP
3,4,Water Content,Not more than 0.5% w/w,Current USP
4,5,Residue on ignition,Not more than 0.10% w/w,Current USP
5,6,Heavy metals,Not more than 0.001%,In-House
6,7,Tartaric acid content by Titrimetry (on anhyd...,Between 36.4% to 38.3% w/w,In-Hosue
7,8,Organic Impurities Procedure-1 A. Phenol Impu...,A. Not more than 0.10% B. Not more than 0.15% ...,Current USP
8,9,Procedure-2 (Enantiomeric Purity) R-Enantiomer,Not more than 0.15%,Current USP
9,10,Assay by HPLC (on anhydrous basis),Between 98.0% to 102.0%,Current USP


In [42]:
LIMITS

,TEST_NAME,SUB_TEST,TEXT_LIMIT,NLT (Not Less Than),NMT (Not More Than)
0,Description,Description,White to Off white powder.,None,None
1,Solubility,Solubility,"Very soluble in water and in methanol, very s...",None,None
2,Identification by IR,Identification by IR,None,None,None
3,Identification by HPLC,Identification by HPLC,None,None,None
4,Water Content,Water Content,Not more than 0.5% w/w,None,0.5% w/w
5,Residue on ignition,Residue on ignition,Not more than 0.10% w/w,None,0.10%
6,Heavy metals,Heavy metals,Not more than 0.001%,None,0.001%
7,Tartaric acid content by Titrimetry (on anhyd...,Tartaric acid content by Titrimetry (on anhyd...,Between 36.4% to 38.3% w/w,36.4% w/w,38.3% w/w
8,Organic Impurities Procedure-1,Phenol Impurity,Not more than 0.10%,None,0.10%
9,Organic Impurities Procedure-1,DPTTA,Not more than 0.15%,None,0.15%
